### Install below libraries in order to run the rest of the notebook
- !pip install openmeteo-requests
- !pip install requests-cache retry-requests numpy pandas

In [1]:
# Import dependencies

import json
import openmeteo_requests # need to install this library with above cell
import requests_cache
from retry_requests import retry
import pandas as pd
import datetime as dt

In [2]:
# Date range
start_date = '2013-01-01'
end_date = '2023-12-31'

# Coordinates
central_park_lat = 40.7896239
central_park_lon = -73.9598939
wash_heights_lat = 40.8417052861612
wash_heights_lon = -73.93934920585686
west_vill_lat = 40.73473345314149
west_vill_lon = -74.00460403083804
midtown_lat = 40.75504999601737
midtown_lon = -73.98400001962062
fin_dist_lat = 40.707717754058876
fin_dist_lon = -74.00783736342065



In [3]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [4]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": [central_park_lat, wash_heights_lat, west_vill_lat, midtown_lat, fin_dist_lat],
	"longitude": [central_park_lon, wash_heights_lon, west_vill_lon, midtown_lon, fin_dist_lon],
	"start_date": start_date,
	"end_date": end_date,
	"daily": ["temperature_2m_max", "temperature_2m_min", "temperature_2m_mean"],
	"temperature_unit": "fahrenheit",
	"timezone": "America/New_York"
}
responses = openmeteo.weather_api(url, params=params)

In [5]:
# Process data. Set empty lists and define neighborhoods variable.
neighborhoods = ['Central Park', 'Washington Heights','West Village','Midtown', 'Financial District']
lat_list = [central_park_lat,wash_heights_lat, west_vill_lat, midtown_lat, fin_dist_lat]
lon_list = [central_park_lon,wash_heights_lon, west_vill_lon, midtown_lon, fin_dist_lon]
min_temp = []
max_temp = []
mean_temp = []
location = []
lat = []
lon = []

# Loop through responses, define lists from each response, assign element of neighborhood to name variable
for x in range(len(responses)):
    response = responses[x]
    daily = response.Daily()
    daily_temperature_2m_max = list(daily.Variables(0).ValuesAsNumpy())
    daily_temperature_2m_min = list(daily.Variables(1).ValuesAsNumpy())
    daily_temperature_2m_mean = list(daily.Variables(2).ValuesAsNumpy())
    name = neighborhoods[x]
    ltd = lat_list[x]
    lng = lon_list[x]
    
    # Loop through number of elements in daily_temperature lists (4017), append each from each list into location, min_temp, max_temp, mean_temp lists
    for y in range(len(daily_temperature_2m_max)):
        location.append(name)
        lat.append(ltd)
        lon.append(lng)
        min_temp.append(daily_temperature_2m_min[y])
        max_temp.append(daily_temperature_2m_max[y])
        mean_temp.append(daily_temperature_2m_mean[y])

In [6]:
# Define the first element in responses to response, then get use .Daily() to get date value
response = responses[0]
daily = response.Daily()

# Create DatetimeIndex object using pd.date_range into date_set
date_set = pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
    )

# Create copy of date_set into date
date = date_set

# Append date_set values into date 4 more times
for x in range(4):
    date = date.append(date_set)

In [7]:
# Create DataFrame
temp_df = pd.DataFrame({
    "Date": date,
    "Location" : location,
    "Latitude": lat,
    "Longitude": lon,
    "Min Temp": min_temp,
    "Max Temp": max_temp,
    "Mean Temp": mean_temp
    })

In [8]:
# Display info
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20085 entries, 0 to 20084
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype              
---  ------     --------------  -----              
 0   Date       20085 non-null  datetime64[ns, UTC]
 1   Location   20085 non-null  object             
 2   Latitude   20085 non-null  float64            
 3   Longitude  20085 non-null  float64            
 4   Min Temp   20085 non-null  float32            
 5   Max Temp   20085 non-null  float32            
 6   Mean Temp  20085 non-null  float32            
dtypes: datetime64[ns, UTC](1), float32(3), float64(2), object(1)
memory usage: 863.2+ KB


In [ ]:
# Preview DataFrame
temp_df.head(20)

In [ ]:
# Save temp_df to CSV file
temp_df.to_csv('Output/Manhattan_Temperatures_2013-2023.csv', index=False)